In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE RecordWildCards #-}

import AdventOfCode

import qualified Data.Attoparsec.ByteString.Char8 as A

input <- dayLines 18

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

data Instr 
    = Snd ValueReg 
    | Set Reg ValueReg
    | Add Reg ValueReg
    | Mul Reg ValueReg
    | Mod Reg ValueReg
    | Rcv Reg
    | Jgz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Snd <$> ("snd" *> A.space *> parseValueReg)
    , Set <$> ("set" *> A.space *> parseReg) <*> (A.space *> parseValueReg)
    , Add <$> ("add" *> A.space *> parseReg) <*> (A.space *> parseValueReg)
    , Mul <$> ("mul" *> A.space *> parseReg) <*> (A.space *> parseValueReg)
    , Mod <$> ("mod" *> A.space *> parseReg) <*> (A.space *> parseValueReg)
    , Rcv <$> ("rcv" *> A.space *> parseReg)
    , Jgz <$> ("jgz" *> A.space *> parseValueReg) <*> (A.space *> parseValueReg)
    ]

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { registers :: M.Map Char Int
    , lastFreq  :: Int
    , counter   :: Int
    } deriving (Eq, Show)

getValueReg registers valueReg = case valueReg of
    Left v -> v
    Right r -> registers M.! r

executeInstr instr ms@MachineState{..} = case instr of
    Snd vr     -> ms{lastFreq = getValueReg registers vr, counter = counter+1}
    Set reg vr -> ms{registers = M.insert reg (getValueReg registers vr) registers, counter = counter+1}
    Add reg vr -> ms{registers = M.adjust (+(getValueReg registers vr)) reg registers, counter = counter+1}
    Mul reg vr -> ms{registers = M.adjust (*(getValueReg registers vr)) reg registers, counter = counter + 1}
    Mod reg vr -> ms{registers = M.adjust (`mod` (getValueReg registers vr)) reg registers, counter = counter + 1}
    Rcv reg    -> ms{counter = counter + 1}
    Jgz vr vr' -> if (getValueReg registers vr) > 0
        then ms{counter = counter + (getValueReg registers vr')}
        else ms{counter = counter + 1}

In [5]:
loop instrs ms@MachineState{..} | counter > V.length instrs || counter < 0 = ms
loop instrs ms@MachineState{..} = case instrs V.! counter of
    Rcv r -> if (registers M.! r) /= 0 then ms else loop instrs (executeInstr (Rcv r) ms)
    curr  -> loop instrs (executeInstr curr ms)

In [6]:
lastFreq $ loop instrVec $ MachineState registerMap 0 0

3423

In [7]:
send msg ms@ConcMachineState{..} = ms{mailbox = mailbox ++ [msg]} 

data LockedState = Running | Waiting deriving (Eq, Show)
data ConcMachineState = ConcMachineState
    { registers   :: M.Map Char Int
    , counter     :: Int
    , mailbox     :: [Int]
    , lockedState :: LockedState
    }

executeInstr instr ms@ConcMachineState{..} = case instr of
    Snd vr     -> ms{counter = counter+1}
    Set reg vr -> ms{registers = M.insert reg (getValueReg registers vr) registers, counter = counter+1}
    Add reg vr -> ms{registers = M.adjust (+(getValueReg registers vr)) reg registers, counter = counter+1}
    Mul reg vr -> ms{registers = M.adjust (*(getValueReg registers vr)) reg registers, counter = counter + 1}
    Mod reg vr -> ms{registers = M.adjust (`mod` (getValueReg registers vr)) reg registers, counter = counter + 1}
    Rcv reg    -> if mailbox == [] 
        then ms{lockedState = Waiting}
        else let
            (msg, mailbox') = (head mailbox, tail mailbox)
            registers' = M.insert reg msg registers
            in ms{registers = registers', mailbox = mailbox', counter = counter + 1, lockedState = Running}
    Jgz vr vr' -> if (getValueReg registers vr) > 0
        then ms{counter = counter + (getValueReg registers vr')}
        else ms{counter = counter + 1}

loop' instrs ms0 ms1 count
    | (lockedState ms0) == Waiting && (lockedState ms1) == Waiting = count
    | (counter ms0) > V.length instrs || (counter ms0) < 0 = count
    | (counter ms1) > V.length instrs || (counter ms1) < 0 = count
loop' instrs ms0 ms1 count = let
    instr0 = instrs V.! (counter ms0)
    instr1 = instrs V.! (counter ms1)
    in case instr0 of
        Snd vr -> let
            value = getValueReg (registers ms0) vr
            ms1' = send value ms1
            ms0' = executeInstr instr0 ms0
            in loop' instrs ms0' ms1' count
        _ -> case instr1 of
            Snd vr -> let
                value = getValueReg (registers ms1) vr
                ms0' = send value ms0
                ms1' = executeInstr instr1 ms1
                in loop' instrs ms0' ms1' (count+1)
            _ -> let
                ms0' = executeInstr instr0 ms0
                ms1' = executeInstr instr1 ms1
                in loop' instrs ms0' ms1' count

initialMs0 = ConcMachineState                 registerMap  0 [] Running
initialMs1 = ConcMachineState (M.insert 'p' 1 registerMap) 0 [] Running

loop' instrVec initialMs0 initialMs1 0

7493